In [1]:
import yfinance as yf
import plotly.graph_objects as go

In [2]:
ticker_symbol = 'AAPL'
start_date = '2021-01-01'
end_date = '2023-01-01'
data = yf.download(ticker_symbol, start=start_date, end=end_date)

/var/folders/_t/pq5rnry94s16k7dtqbwrj1nr0000gn/T/ipykernel_47368/1264610966.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_symbol, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


In [3]:
data

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2021-01-04,125.974464,130.062961,123.394814,129.975354,143301900
2021-01-05,127.531960,128.242590,125.020451,125.468246,97664900
2021-01-06,123.239067,127.570935,123.024906,124.329336,155088000
2021-01-07,127.444397,128.135555,124.465635,124.952361,109578200
2021-01-08,128.544373,129.108977,126.772682,128.914274,105158200
...,...,...,...,...,...
2022-12-23,129.900284,130.451959,127.713276,128.974252,63814900
2022-12-27,128.097488,129.456984,126.806960,129.427431,69007800


In [4]:
# Create the plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Close Price'))
fig.update_layout(title=f'{ticker_symbol} Stock Prices from Yahoo Finance',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  template='simple_white',
                  width=900,
                  height=500) 
fig.show()

# Stationary Time Series

A time series is a time series whose statistical properties do not change over time.

A time series is stationary if its:

Mean is constant

Variance is constant

Autocovariance (relationship between points at different lags) is constant over time


A time series 𝑋𝑡 is strictly stationary if the joint probability distribution of

(Xt1​​,Xt2​​,…,Xtk​​)

is the same as the distribution of

(Xt1​+τ​,Xt2​+τ​,…,Xtk​+τ​)

for any time shift 𝜏.
i.e.
if you slide the time window, the statistical behavior stays the same.

In [7]:
import plotly.express as px
import pandas as pd
import numpy as np 
from statsmodels.tsa.stattools import adfuller

In [8]:
data = pd.read_csv('air_passengers.csv')
data

,Month,#Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390
